In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from tqdm import tqdm
import gc
import random
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
PATH = './data/'

train_err  = pd.read_csv(PATH+'train_err_data.csv')
train_qua = pd.read_csv(PATH+'train_quality_data.csv')
train_prob = pd.read_csv(PATH+'train_problem_data.csv')

test_err  = pd.read_csv(PATH+'test_err_data.csv')
test_qua  = pd.read_csv(PATH+'test_quality_data.csv')

sample_submssion = pd.read_csv(PATH+'sample_submission.csv')

In [3]:
id_error = train_err[['user_id','errtype']].values

train_user_id_max = 24999
train_user_id_min = 10000
train_user_number = 15000

error = np.zeros((train_user_number,42))

for person_idx, err in tqdm(id_error):
    error[person_idx - train_user_id_min,err - 1] += 1
error.shape

100%|██████████████████████████████████████████████████████████████████| 16554663/16554663 [00:49<00:00, 332356.20it/s]


(15000, 42)

In [4]:
problem = np.zeros(15000)

problem[train_prob.user_id.unique()-10000] = 1 
problem.shape

(15000,)

In [5]:
test_user_id_max = 44998
test_user_id_min = 30000
test_user_number = 14999

id_error = test_err[['user_id','errtype']].values
test_error = np.zeros((test_user_number,42))
for person_idx, err in tqdm(id_error):
    test_error[person_idx - test_user_id_min,err - 1] += 1
test_error = test_error.reshape(test_error.shape[0],-1)
print(test_error.shape)

100%|██████████████████████████████████████████████████████████████████| 16532648/16532648 [00:51<00:00, 322286.68it/s]

(14999, 42)


In [29]:
import tensorflow as tf
from tensorflow.keras import datasets
from tensorflow.keras import layers
from sklearn.metrics import *

tf.random.set_seed(42)

In [7]:
from sklearn.model_selection import train_test_split

train_X, test_X, train_y, test_y = train_test_split(error, problem, test_size=0.25, random_state=42)

train_X.shape, test_X.shape, train_y.shape, test_y.shape

((11250, 42), (3750, 42), (11250,), (3750,))

In [39]:
model = tf.keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(error.shape)),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

In [40]:
model.compile(loss='binary_crossentropy', optimizer='rmsprop')
model.fit(train_X, train_y, epochs=50, batch_size=48, validation_split=0.25)
model.evaluate(test_X, test_y)

Epoch 1/50
176/176 [==============================] - 1s 2ms/step - loss: 2.5035 - val_loss: 4.1567
Epoch 2/50
176/176 [==============================] - 0s 2ms/step - loss: 0.9660 - val_loss: 3.4181
Epoch 3/50
176/176 [==============================] - 0s 1ms/step - loss: 0.6777 - val_loss: 3.3163
Epoch 4/50
176/176 [==============================] - 0s 2ms/step - loss: 0.4932 - val_loss: 2.7786
Epoch 5/50
176/176 [==============================] - 0s 1ms/step - loss: 0.5277 - val_loss: 3.3747
Epoch 6/50
176/176 [==============================] - 0s 2ms/step - loss: 0.6198 - val_loss: 2.7252
Epoch 7/50
176/176 [==============================] - 0s 2ms/step - loss: 0.4341 - val_loss: 3.0540
Epoch 8/50
176/176 [==============================] - 0s 1ms/step - loss: 0.4105 - val_loss: 4.5914
Epoch 9/50
176/176 [==============================] - 0s 2ms/step - loss: 0.3660 - val_loss: 3.2080
Epoch 10/50
176/176 [==============================] - 0s 2ms/step - loss: 0.4851 - val_loss: 5.7644

1.4428296089172363

In [41]:
pred = model.predict_proba(test_X)
pred
print(roc_auc_score(test_y, pred))

0.7242500914522619


In [44]:
model.compile(loss='binary_crossentropy', optimizer='rmsprop')
model.fit(error, problem, epochs=50, batch_size=48, validation_split=0.25)
sample_submssion['problem'] = model.predict_proba(test_error)

Epoch 1/50
235/235 [==============================] - 1s 2ms/step - loss: 1.2812 - val_loss: 0.6244
Epoch 2/50
235/235 [==============================] - 0s 1ms/step - loss: 0.4947 - val_loss: 0.5824
Epoch 3/50
235/235 [==============================] - 0s 1ms/step - loss: 0.4950 - val_loss: 0.6174
Epoch 4/50
235/235 [==============================] - 0s 1ms/step - loss: 0.6155 - val_loss: 0.5844
Epoch 5/50
235/235 [==============================] - 0s 1ms/step - loss: 0.5065 - val_loss: 0.5824
Epoch 6/50
235/235 [==============================] - 0s 1ms/step - loss: 0.5464 - val_loss: 0.5660
Epoch 7/50
235/235 [==============================] - 0s 1ms/step - loss: 0.5565 - val_loss: 0.5592
Epoch 8/50
235/235 [==============================] - 0s 1ms/step - loss: 0.7026 - val_loss: 0.5456
Epoch 9/50
235/235 [==============================] - 0s 2ms/step - loss: 0.4923 - val_loss: 0.5404
Epoch 10/50
235/235 [==============================] - 0s 1ms/step - loss: 0.5071 - val_loss: 0.5912

In [45]:
sample_submssion.to_csv("./submission/submission_20210114-2.csv", index = False)
sample_submssion

,user_id,problem
0,30000,0.996796
1,30001,0.198434
2,30002,0.244801
3,30003,0.402899
4,30004,0.991724
...,...,...
14994,44994,0.247038
14995,44995,0.214953
14996,44996,1.000000
14997,44997,0.155185
